In [3]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import matplotlib.pyplot as plt
from sklearn import svm
import numpy as np
from sklearn import linear_model
import pandas as pd 
from ydata_profiling import ProfileReport

In [17]:
file_path_train = "C:/Users/jeyas/OneDrive/Documents/CSE158/HeartAttackPrediction/Datasets/diabetes_train.csv"
file_path_test = "C:/Users/jeyas/OneDrive/Documents/CSE158/HeartAttackPrediction/Datasets/diabetes_test.csv"
dataTrain = pd.read_csv(file_path_train) #, index_col = "Unnamed:0")
dataTest = pd.read_csv(file_path_test)
dataTest.rename(columns = {"Unnamed: 0": "index"}, inplace = True)
dataTrain.rename(columns = {"Unnamed: 0": "index"}, inplace = True)

In [18]:
dataTrain

,index,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,0,Female,80.0,0,1,never,25.19,6.6,140,0
1,1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,2,Male,28.0,0,0,never,27.32,5.7,158,0
3,3,Female,36.0,0,0,current,23.45,5.0,155,0
4,4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...,...
89995,89995,Female,7.0,0,0,No Info,16.44,4.0,90,0
89996,89996,Female,25.0,0,0,not current,28.30,6.6,140,0
89997,89997,Female,58.0,1,0,No Info,42.90,7.0,280,1
89998,89998,Male,73.0,0,0,never,27.32,4.0,126,0


In [32]:
def feature(row, dataTrain = dataTrain):
    feat = dataTrain.loc[row]['HbA1c_level']
    return [1] + [feat]

In [33]:
X_train = [feature(row) for row in range(len(dataTrain))]
Y_train = [dataTrain.loc[row]['diabetes'] for row in range(len(dataTrain))]
theta,residuals,rank,s = np.linalg.lstsq(X_train, Y_train)
mse = residuals/len(dataTrain)

C:\Users\jeyas\AppData\Local\Temp\ipykernel_28364\3781606520.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  theta,residuals,rank,s = np.linalg.lstsq(X_train, Y_train)


In [35]:
X_test = [feature(row) for row in range(len(dataTest))]
Y_test = [dataTest.loc[row]['diabetes'] for row in range(len(dataTest))]

In [37]:
def predY(x,theta=theta):
    return theta[0]*x[0] + theta[1]*x[1]

In [38]:
pred = [predY(row) for row in X_test]

In [40]:
def accuracy(predictions, y):
    correct = predictions == y # Binary vector indicating which predictions were correct
    return sum(correct) / len(correct)

Correlation with HbA1c Levels

In [46]:
dataTest

,index,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,90000,Male,56.0,1,0,never,31.77,6.1,140,0
1,90001,Female,63.0,0,0,not current,27.32,6.6,90,0
2,90002,Female,68.0,0,0,never,27.32,8.2,260,1
3,90003,Male,13.0,0,0,No Info,27.32,4.8,158,0
4,90004,Male,18.0,0,0,never,27.32,6.0,145,0
...,...,...,...,...,...,...,...,...,...,...
9995,99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
9996,99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
9997,99997,Male,66.0,0,0,former,27.83,5.7,155,0
9998,99998,Female,24.0,0,0,never,35.42,4.0,100,0


In [63]:
dataTrain["HbA1c_level"].min()

3.5

In [59]:
best_accuracy = 0 
best_threshold = 0
for threshold in np.arange(0, 4, 0.5):
    ypred = [1 if dataTrain.loc[row]['HbA1c_level'] > threshold else 0 for row in dataTrain.index]
    correct = [1 if dataTrain.loc[row]['diabetes'] == pred else 0 for row,pred in zip(ypred, dataTrain.index)]
    accuracy = sum(correct)/len(correct)
    if accuracy > best_accuracy:
        best_accuracy = accuracy 
        best_threshold = threshold 
    print(threshold)


0.0
0.5
1.0
1.5
2.0
2.5
3.0
3.5


In [61]:
best_threshold

0.0

In [ ]:
ypred = [if dataTest.loc[row]['HbA1c_level'] for row in dataTest.index]